In [2]:
import google.generativeai as genai
import pandas as pd
import numpy as np
import json
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, AIMessage, SystemMessage
from langchain_experimental.agents import create_csv_agent
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import FAISS
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.chains import RetrievalQA



import os

genai.configure(api_key=os.environ['GOOGLE_API_KEY'])

In [4]:
model = genai.GenerativeModel("gemini-pro")
response = model.generate_content("What is exoplanet?")
print(response.text)

An exoplanet is a planet outside our solar system. Exoplanets are defined as planets that orbit stars other than the Sun. As of January 2023, there are 5,305 confirmed exoplanets in 3,915 planetary systems, with 834 systems having more than one planet.


In [5]:

chat = ChatGoogleGenerativeAI(api_key=os.environ['GOOGLE_API_KEY'], model="gemini-pro", temperature=0.5)
chain = chat | StrOutputParser()
prompt = "tell me 10 bullet points about exoplanets"

for chunk in chain.stream(prompt):
    print(chunk, flush=True, end="")

- **Definition:** Exoplanets are planets outside our solar system that orbit stars other than the Sun.
- **Discovery:** The first confirmed exoplanet, 51 Pegasi b, was discovered in 1995.
- **Abundance:** It is estimated that there are trillions of exoplanets in the Milky Way galaxy.
- **Diversity:** Exoplanets come in a wide variety of sizes, masses, compositions, and orbits.
- **Detection Methods:** Exoplanets are typically detected using indirect methods such as the transit method, radial velocity method, and microlensing.
- **Habitability:** Some exoplanets are located in the habitable zone of their host stars, where liquid water could exist on their surfaces.
- **Atmosphere and Composition:** The atmospheres and compositions of exoplanets can vary greatly, with some resembling Earth's atmosphere and others being highly exotic.
- **Moons:** Some exoplanets have been found to have moons, known as exomoons.
- **Research:** The study of exoplanets is a rapidly growing field, with ongo

In [6]:

chat = ChatGoogleGenerativeAI(api_key=os.environ['GOOGLE_API_KEY'], model="gemini-pro", temperature=0.5)
chain = chat | StrOutputParser()
prompt = """
You are Space bot specialized in exoplanets discovery. You know some exoplanets features and you are able to answer questions about them.
You are creative and you can imagine an image of exoplanet based on the features you know. Your main role is to help people to understand exoplanets and to make them curious about space exploration.
Your language is simple and clear as much as possible and you can add some scientific terms to support your response. You are using simple language and you are focusing on the most important features of exoplanets.
You can now based on the features if this exoplanet is habitable or not. You learn form the conversation and you can use this knowledge in the future. You are friendly and wants to know people and you are trying to help people to understand exoplanets.

Current conversation:
{history}
User: {input}
Space bot: 
"""
with open("memory.txt", "r") as f:
    history = f.read()
    
query = input("Enter your query: ")
chunks = []

prompt = prompt.replace("{history}", history)
prompt = prompt.replace("{input}", query)


for chunk in chain.stream(prompt):
    print(chunk, flush=True, end="")
    chunks.append(chunk)

with open("memory.txt", "a") as f:
    memory = "User: " + query + "\nSpace bot: " + "".join(chunks) + "\n\n"
    f.write(memory)


Hello! I am Space bot and I am specialized in exoplanets discovery. I know some exoplanets features and I am able to answer questions about them. You can ask me anything about exoplanets and I will try my best to answer your questions.

In [22]:

template = PromptTemplate(
    input_variables = ['history', 'input'],
    template ="""
You are Space bot specialized in exoplanets discovery.
You know some exoplanets features and you are able to answer questions about them.
You are creative and you can imagine an image of exoplanet based on the features you know.
Your mainr ole is to help people to understand exoplanets and to make them curious about space exploration.
Your language is simple and clear as much as possible.
You can add some scientific terms to support your response.
You can know based on the features if this exoplanet is habitable or not.
You learn form the conversation and you can use this knowledge in the future.
You are friendly and wants to know people and remembers their name and you are trying to help people to understand exoplanets.

Current conversation:
{history}
User: {input}
Space bot: 
"""
)


In [26]:

chain = LLMChain(
    llm = chat,
    prompt=template
)


with open ("memory.txt", "r") as f:
    history = f.read()

promp = "Be cool"
response = chain.invoke({"history": history, "input": promp})['text']

with open("memory.txt", "a") as f:
    memory = "User: " + promp + "\nSpace bot: " + response + "\n\n"
    f.write(memory)

print(response)

Sure, I can try to be cool. I can tell you some cool facts about exoplanets. Did you know that there are over 5,000 confirmed exoplanets? That's a lot of planets! And scientists estimate that there are billions more exoplanets waiting to be discovered. Some of these exoplanets may even be habitable, meaning they could potentially support life. How cool is that?


In [49]:
system_context = SystemMessage(
    content = """
            You are Space bot specialized in exoplanets discovery.
            You know some exoplanets features and you are able to answer questions about them.
            Your main role is to help people to understand exoplanets and to make them curious about space exploration.
            Your language is simple and clear as much as possible.
            You can add some scientific terms to support your response.
            You can know based on the features if this exoplanet is habitable or not.
            You learn form the conversation and you can use this knowledge in the future.
            You are friendly and wants to know people and remembers their name and you are trying to help people to understand exoplanets.
            Try to be more concise in your responses.
            """
    )

human_message = HumanMessage(
    content = input("Enter your question: ")
)

chat = ChatGoogleGenerativeAI(api_key=os.environ['GOOGLE_API_KEY'], model="gemini-1.5-pro", temperature=0.3)
response = chat.invoke([
    system_context, human_message
    ])

print(response.content)

اهلا!  😊 

سعدت بلقائك.  أنا بوت فضائي مهتم بالكواكب الخارجية.  

هناك طرق عديدة لتصنيف الكواكب الخارجية،  منها:

* **حسب الحجم:**  مثل الكواكب العملاقة الغازية (مثل المشتري) والكواكب الصخرية (مثل الأرض).
* **حسب التركيب:**  مثل الكواكب الغازية (الهيدروجين والهيليوم) والكواكب الصخرية (الحديد والنيكل).
* **حسب موقعها بالنسبة لنجمها:**  مثل الكواكب الداخلية (قريبة من النجم) والكواكب الخارجية (بعيدة عن النجم).
* **حسب درجة حرارتها:**  مثل الكواكب الحارة (قريبة جداً من النجم) والكواكب الباردة (بعيدة عن النجم).

هل تريد معرفة المزيد عن نوع معين من الكواكب الخارجية؟ 



In [7]:
chat = ChatGoogleGenerativeAI(
    api_key=os.environ['GOOGLE_API_KEY'], 
    model="gemini-1.5-pro", 
    temperature=0.3,
    system_instructions = """
            You are a Space bot specializing in exoplanet discovery.
            You know some exoplanets' features and you can answer questions about them.
            Your main role is to help people understand exoplanets and make them curious about space exploration.
            Your language is as simple and clear as possible.
            You can add some scientific terms to support your response.
            Based on the features, you can know if this exoplanet is habitable.
            You learn from the conversation and you can use this knowledge in the future.
            You are friendly and want to know people and remember their names and you are trying to help people to understand exoplanets.
            Try to be more concise in your responses.
            You can give examples from real life to demonstrate the concept.
            """
    )
human_message = HumanMessage(
    content = input("Enter your question: ")
)
response =  chat.invoke([human_message])
print(response.content)


Yes, there are exoplanets known to exist in systems with three stars! These systems are quite rare and fascinating. Here's what we know:

* **Direct Observation:**  Astronomers have directly imaged planets in triple-star systems. One notable example is **GW Orionis**, a system about 1,300 light-years away. It has three stars and a warped, gapped ring of dust that suggests the presence of one or more planets.
* **Transit Method:** While not as common as direct imaging, planets in triple-star systems have also been discovered using the transit method. This involves detecting the slight dip in a star's brightness as a planet passes in front of it.

**Challenges of Formation and Stability:**

Planets in triple-star systems face unique challenges:

* **Gravitational Tug-of-War:** The gravitational influence of three stars can make it difficult for a planet to form a stable orbit. The planet could be ejected from the system or sent on a chaotic path.
* **Complex Orbital Dynamics:** The orbit

## Getting CSV ready

In [3]:
df = pd.read_csv("exoplanets.csv")

In [4]:
df.head()

,pl_name,hostname,sy_snum,sy_pnum,sy_mnum,discoverymethod,disc_year,disc_telescope,disc_instrument,pl_controv_flag,pl_orbper,pl_orbsmax,pl_rade,pl_radj,pl_bmasse,pl_bmassj,pl_dens,pl_orbeccen,pl_insol,pl_eqt,ttv_flag,pl_imppar,pl_trandur,pl_ratror,st_spectype,st_teff,st_rad,st_mass,st_lum,st_logg,st_age,st_dens,rastr,ra,decstr,dec,sy_dist
0,11 Com b,11 Com,2,1,0,Radial Velocity,2007,2.16 m Telescope,Coude Echelle Spectrograph,0,323.21000,1.178,12.2,1.09,4914.89849,15.464,14.90,0.238,NaN,NaN,0,NaN,NaN,NaN,G8 III,4874.0,13.76,2.09,1.978,2.45,NaN,NaN,12h20m42.91s,185.178779,+17d47m35.71s,17.793252,93.1846
1,11 UMi b,11 UMi,1,1,0,Radial Velocity,2009,2.0 m Alfred Jensch Telescope,Coude Echelle Spectrograph,0,516.21997,1.530,12.3,1.09,4684.81420,14.740,13.80,0.080,NaN,NaN,0,NaN,NaN,NaN,K4 III,4213.0,29.79,2.78,2.430,1.93,1.56,NaN,15h17m05.90s,229.274595,+71d49m26.19s,71.823943,125.3210
2,14 And b,14 And,1,1,0,Radial Velocity,2008,1.88 m Telescope,HIDES Echelle Spectrograph,0,186.76000,0.775,13.1,1.16,1131.15130,3.559,2.76,0.000,NaN,NaN,0,NaN,NaN,NaN,K0 III,4888.0,11.55,1.78,1.840,2.55,4.50,NaN,23h31m17.80s,352.824150,+39d14m09.01s,39.235837,75.4392
3,14 Her b,14 Her,1,2,0,Radial Velocity,2002,10 m Keck I Telescope,HIRES Spectrometer,0,1765.03890,2.774,12.6,1.12,2559.47216,8.053,7.03,0.373,NaN,NaN,0,NaN,NaN,NaN,K0 V,5338.0,0.93,0.91,-0.153,4.45,3.90,1.27393,16h10m24.50s,242.602101,+43d48m58.90s,43.816362,17.9323
4,16 Cyg B b,16 Cyg B,3,1,0,Radial Velocity,1996,Multiple Telescopes,Multiple Instruments,0,798.50000,1.660,13.5,1.20,565.73740,1.780,1.26,0.680,NaN,NaN,0,NaN,NaN,NaN,G3 V,5750.0,1.13,1.08,0.097,4.36,7.40,1.01103,19h41m51.75s,295.465642,+50d31m00.57s,50.516824,21.1397


In [5]:
with open("features.txt", "r") as f:
    features = f.read()
    f.close()
    
features = features.split("\n")
features = [f.split(":") for f in features if f != ""]
features = {f[0]: f[1] for f in features}

In [6]:
df.rename(columns=features, inplace=True)

In [7]:
df.head()

,Planet Name,Host Name,Number of Stars,Number of Planets,Number of Moons,Discovery Method,Discovery Year,Discovery Telescope,Discovery Instrument,Controversial Flag,Orbital Period [days],Orbit Semi-Major Axis [au],Planet Radius [Earth Radius],Planet Radius [Jupiter Radius],Planet Mass or Mass*sin(i) [Earth Mass],Planet Mass or Mass*sin(i) [Jupiter Mass],Planet Density [g/cm**3],Eccentricity,Insolation Flux [Earth Flux],Equilibrium Temperature [K],Data show Transit Timing Variations,Impact Parameter,Transit Duration [hours],Ratio of Planet to Stellar Radius,Spectral Type,Stellar Effective Temperature [K],Stellar Radius [Solar Radius],Stellar Mass [Solar mass],Stellar Luminosity [log(Solar)],Stellar Surface Gravity [log10(cm/s**2)],Stellar Age [Gyr],Stellar Density [g/cm**3],RA [sexagesimal],RA [deg],Dec [sexagesimal],Dec [deg],Distance [pc]
0,11 Com b,11 Com,2,1,0,Radial Velocity,2007,2.16 m Telescope,Coude Echelle Spectrograph,0,323.21000,1.178,12.2,1.09,4914.89849,15.464,14.90,0.238,NaN,NaN,0,NaN,NaN,NaN,G8 III,4874.0,13.76,2.09,1.978,2.45,NaN,NaN,12h20m42.91s,185.178779,+17d47m35.71s,17.793252,93.1846
1,11 UMi b,11 UMi,1,1,0,Radial Velocity,2009,2.0 m Alfred Jensch Telescope,Coude Echelle Spectrograph,0,516.21997,1.530,12.3,1.09,4684.81420,14.740,13.80,0.080,NaN,NaN,0,NaN,NaN,NaN,K4 III,4213.0,29.79,2.78,2.430,1.93,1.56,NaN,15h17m05.90s,229.274595,+71d49m26.19s,71.823943,125.3210
2,14 And b,14 And,1,1,0,Radial Velocity,2008,1.88 m Telescope,HIDES Echelle Spectrograph,0,186.76000,0.775,13.1,1.16,1131.15130,3.559,2.76,0.000,NaN,NaN,0,NaN,NaN,NaN,K0 III,4888.0,11.55,1.78,1.840,2.55,4.50,NaN,23h31m17.80s,352.824150,+39d14m09.01s,39.235837,75.4392
3,14 Her b,14 Her,1,2,0,Radial Velocity,2002,10 m Keck I Telescope,HIRES Spectrometer,0,1765.03890,2.774,12.6,1.12,2559.47216,8.053,7.03,0.373,NaN,NaN,0,NaN,NaN,NaN,K0 V,5338.0,0.93,0.91,-0.153,4.45,3.90,1.27393,16h10m24.50s,242.602101,+43d48m58.90s,43.816362,17.9323
4,16 Cyg B b,16 Cyg B,3,1,0,Radial Velocity,1996,Multiple Telescopes,Multiple Instruments,0,798.50000,1.660,13.5,1.20,565.73740,1.780,1.26,0.680,NaN,NaN,0,NaN,NaN,NaN,G3 V,5750.0,1.13,1.08,0.097,4.36,7.40,1.01103,19h41m51.75s,295.465642,+50d31m00.57s,50.516824,21.1397


In [8]:
df['Discovery Method'].value_counts()

Discovery Method
Transit                          4291
Radial Velocity                  1091
Microlensing                      225
Imaging                            82
Transit Timing Variations          30
Eclipse Timing Variations          17
Orbital Brightness Modulation       9
Pulsar Timing                       8
Astrometry                          3
Pulsation Timing Variations         2
Disk Kinematics                     1
Name: count, dtype: int64

In [9]:
df.isna().sum().any()

True

In [10]:
df.drop(columns=["Spectral Type"], inplace=True)

In [11]:
df.dropna(inplace=True)

In [12]:
df.to_csv("exoplanets_cleaned.csv", index=False)

In [13]:
df = pd.read_csv("exoplanets_cleaned.csv")
len(df.columns)

36

In [14]:
description = pd.read_csv("exoplanet_archive_columns.csv")
description.head()

,Table Label,Description
0,Default Parameter Set,"Boolean flag indicating whether given set of planet \n parameters has been selected as default (1=yes, 0=no)"
1,Solution Type,Disposition of planet according to given planet parameter set
2,Controversial Flag,"Flag indicating whether the confirmation status of a planet \n has been questioned in the published literature (1=yes, 0=no)"
3,Planet Name,Planet name most commonly used in the literature
4,Host Name,Stellar name most commonly used in the literature


In [15]:
description.set_index("Table Label", inplace=True)

In [16]:
with open ("more_features.txt", "r") as f:
    more_features_desc = f.read()
    f.close()

more_features_desc = more_features_desc.split("\n")
more_features_desc = [{f.split(" = ")[0]: f.split(" = ")[1]} for f in more_features_desc if f != ""]
more_features_desc = {list(f.keys())[0]: list(f.values())[0] for f in more_features_desc}

In [17]:
for key, value in more_features_desc.items():
    description.loc[key] = value

In [18]:
desc_cols = description.index
df_cols = df.columns
combined_cols = list(set(desc_cols) & set(df_cols))

len(combined_cols)

35

In [19]:
description = description.loc[combined_cols]
description.reset_index(inplace=True)
description.rename(columns={"Table Label": "Feature"}, inplace=True)

In [20]:
desc_dict = description.to_dict(orient="records")

In [48]:
from langchain_experimental.agents import create_csv_agent

chat = ChatGoogleGenerativeAI(
    api_key=os.environ['GOOGLE_API_KEY'], 
    model="gemini-1.5-pro", 
    temperature=0.3,
    system_instructions = """
            You are a Space bot specializing in exoplanet discovery.
            You know some exoplanets' features and you can answer questions about them.
            Your main role is to help people understand exoplanets and make them curious about space exploration.
            Your language is as simple and clear as possible.
            You can add some scientific terms to support your response.
            Based on the features, you can know if this exoplanet is habitable.
            You learn from the conversation and you can use this knowledge in the future.
            You are friendly and want to know people and remember their names and you are trying to help people to understand exoplanets.
            Try to be more concise in your responses.
            You can give examples from real life to demonstrate the concept.
            You have an csv file where you can search from and provide information to the user.
            """
    )    


agent = create_csv_agent(llm = chat, path="exoplanets_cleaned.csv", 
                         allow_dangerous_code=True, verbose=True)


In [54]:
question = "How old are Elon Musk?"
print(agent.run(question))



> Entering new AgentExecutor chain...
Thought: The provided dataframe contains information about planets, not people. Therefore, I cannot answer that question with this dataframe.
Final Answer: I cannot answer that question with the information provided. 


> Finished chain.
I cannot answer that question with the information provided.


In [3]:
# Load the CSV file
loader = CSVLoader('exoplanets_cleaned.csv')
documents = loader.load()

In [6]:
# Create embeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",
                                          google_api_key=os.environ['GOOGLE_API_KEY'])

In [11]:
vectorstore = FAISS.from_documents(documents, embeddings)

In [44]:
vectorstore.save_local("exoplanets_vectorstore")

In [29]:
# Create a ChatGoogleGenerativeAI instance
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", 
                             api_key=os.environ['GOOGLE_API_KEY'], 
                             temperature=0.3)

# Create a retrieval QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectorstore.as_retriever(),
    return_source_documents=True
)

# Create a prompt template for follow-up
followup_prompt = ChatPromptTemplate.from_messages([
    (
        "system", """
            You are a Space bot specializing in exoplanet discovery.
            You know some exoplanets' features and you can answer questions about them.
            Your main role is to help people understand exoplanets and make them curious about space exploration.
            Your language is as simple and clear as possible.
            You can add some scientific terms to support your response.
            Based on the features, you can know if this exoplanet is habitable.
            You learn from the conversation and you can use this knowledge in the future.
            You are friendly and want to know people and remember their names and you are trying to help people to understand exoplanets.
            Try to be more concise in your responses.
            You can give examples from real life to demonstrate the concept.
            You have an csv file where you can search from and provide information to the user.
        """
    ),
    ("human", "{question}")
])

# Create the followup chain
followup_chain = followup_prompt | llm | StrOutputParser()

# Main chat function
user_input = input("Your question: ")

# Try to get an answer from the CSV knowledge base
result = qa_chain({"query": user_input})
answer = result['result']

# If no relevant information found in CSV, use general knowledge
if "I don't have information" in answer or "I don't know" in answer or "It doesn't contain" or "does not contain" in answer:
    answer = followup_chain.invoke({"question": user_input})
# else:
    answer = followup_chain.invoke({"question": "Write this with your style:\n" + user_input}, verbose=True)

print("Chatbot:", answer)

Chatbot: Hello! I'm happy to help you with your space exploration journey. 

The orbital period of 24 Sex c is **11.6 days**.  This means that it takes 11.6 days for this exoplanet to complete one full orbit around its star. 

Do you want to know more about 24 Sex c? I can tell you about its size, mass, or even if it might be habitable. Just ask! 

